In [1]:
# %pip install langchain fastembed qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 702.8 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 2.7 MB/s eta 0:00:0000:0100:01
  Using cached snowballstemmer-2.2.0-py2.py3-none-any.whl (93 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for PyStemmer: filename=PyStemmer-2.2.0.1-cp310-cp310-macosx_11_0_arm64.whl size=172717 sha256=71ede1728c9acc96edd1b728df8cc19f46859dab8a48f8d6d784493e3267d017
  Stored in directory: /Users/devanshu/Library/Caches/pip/wheels/45/7d/2c/a7ebb8319e01acc5306fa1f8558bf24063d6cec2c02de330c9
Successfully built PyStemmer
  Attempting uninstall: pillow
    Found existing installation: pillow 10.2.0
    Uninstalling pillow-10.2.0:
      Successfully uninstalled pillow-10.2.0
ERROR: pip's dependency resolver does not currently take into

In [3]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from qdrant_client import QdrantClient
from qdrant_client.http import models


import uuid
import time
from typing import List
from fastembed import TextEmbedding
from qdrant_client.http.models import PointStruct, SearchParams

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain_groq import ChatGroq


In [16]:
qdrant_uri = "https://14ea86c5-419c-41e5-b82b-18eec7c016e2.us-east4-0.gcp.cloud.qdrant.io:6333"
qdrant_api = "cfhdTR3705TH3iU78F6WY6dQGmjWuO2ofiLzuyr4LR3bF8zf4VZ_Rw"
groq_key = "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"

In [6]:
llm = ChatGroq(groq_api_key=groq_key,
               model='llama3-8b-8192',
               temperature=0.2) # set temperature by your own

In [7]:
data = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k", split="train")
data = data.to_pandas()
data.head()

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

,instruction,input,output
0,"If you are a doctor, please answer the medical...",I woke up this morning feeling the whole room ...,"Hi, Thank you for posting your query. The most..."
1,"If you are a doctor, please answer the medical...",My baby has been pooing 5-6 times a day for a ...,Hi... Thank you for consulting in Chat Doctor....
2,"If you are a doctor, please answer the medical...","Hello, My husband is taking Oxycodone due to a...","Hello, and I hope I can help you today.First, ..."
3,"If you are a doctor, please answer the medical...",lump under left nipple and stomach pain (male)...,HI. You have two different problems. The lump ...
4,"If you are a doctor, please answer the medical...",I have a 5 month old baby who is very congeste...,Thank you for using Chat Doctor. I would sugge...


In [8]:
MAX_ROWS = 1000
OUTPUT = "output"
subset_data = data.head(MAX_ROWS)

In [9]:
client = QdrantClient(
    qdrant_uri,
    api_key=qdrant_api
)


chunks = subset_data[OUTPUT].to_list()

# Add data to defined collection
client.add(
   collection_name='ChatDoctor',
   documents=chunks
)

100%|██████████| 77.7M/77.7M [00:05<00:00, 13.1MiB/s]


['710592a7e1e54112bfcdaef5eadd0801',
 'b26622a3ecc04a05b5f4f0ede2929cc4',
 '876bedc8a6954b00a56a04e9601eab73',
 'a70f54e68226451ba2d58a76f08391d7',
 '13bdd7fa9971463c85fa787db3b769ed',
 '397f96c3b93c45d79462c4f1a995a5db',
 '932c5237d77e4502af0b4a4ade7e5aa3',
 '4bcf1f06d8fa48f282fcdf0023abe908',
 'ff85567104eb4bd2b216f7f44c7889d3',
 'b7de6dff0c5242d6a6f2fd8d0f11848a',
 '38454e5aaaa240a2ac5b1c51ec0478f6',
 '6e9595e57305438da8619756835dcf24',
 'b3560cc32b504913ab18bb10b2171786',
 'cc8f73c768b84c6caed228ca3ff42fa0',
 'e5e813b93f534164bda428ef7bac1212',
 '7b1e4dc9599248c2ae970c02106c26ef',
 'e45fdcf4f7164c0c89895f7ad8d9d41a',
 'f8aef91870e24a6ea0656dcf06d3acbf',
 '7b8867e24a2246af9b4fdd49bee19029',
 '44e22870d2fa439989c80b5ffb740ddb',
 'e4498ff72a2b4b179ffd342dfbd03ffe',
 '37da9eb22ddf430a9897638233409ea3',
 '28f4504d4313448e9e1854d3c37127c8',
 '637c2f74c1d54c6bb578f6c0cb672220',
 '43711924e8264bfba7fc6fa620a569f3',
 'd26f96392e1840798855c33aaa862ed8',
 '19cfead9ed7b44e09005ea004ac2d6eb',
 

In [18]:
threshold = 0.5

In [19]:
class QdrantVectorStore():
    def __init__(self, qdrant_uri, qdrant_api, threshold):
        self.encoder = TextEmbedding(model_name="BAAI/bge-small-en-v1.5")
        self.client = QdrantClient()
        self.db_collection_name = "ChatDoctor"
        
        self.db_client = QdrantClient(
            qdrant_uri,
            api_key=qdrant_api
        )
        
        self.euclidean_threshold = threshold
   
    def get_embedding(self, question):
        embedding = list(self.encoder.embed(question))[0]
        return embedding
   
    def query_database(self, query_text):
        result = self.db_client.query(
            query_text=query_text,
            limit=3,
            collection_name=self.db_collection_name
        )
        return result
   
    def query(self, question):
        start_time = time.time()
        
        db_results = self.query_database(question)
        
        if db_results:
            response_text = db_results[0].document
            print('Retrieval without cache.')
            elapsed_time = time.time() - start_time
            print(f"Time taken: {elapsed_time:.3f} seconds")
            return response_text
        
        print("No answer found in Database")
        elapsed_time = time.time() - start_time
        print(f"Time taken: {elapsed_time:.3f} seconds")
        return "No answer available"


In [21]:
vector_db = QdrantVectorStore(qdrant_uri, qdrant_api, threshold)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [44]:
def chat_query(question):
    context = vector_db.query(question)
   
    prompt_template = """Answer the question with given context for the medical question. Please ensure that the answer is relevant to the question.
                In case the original context is not relevant, please answer the question as per the question only.
                {context}
           
                Question: {question}"""
    prompt = PromptTemplate.from_template(prompt_template)


    chain = prompt | llm | StrOutputParser()
   
    result = chain.invoke({"question":question, "context":context})
    # print("\n\n")
    return result

In [47]:
question_2 = "I have a sore throat."
result = chat_query(question_2)

Retrieval without cache.
Time taken: 1.092 seconds


In [48]:
from IPython.display import Markdown
Markdown(result)

Based on the given context, it seems that your sore throat might be accompanied by other symptoms such as a runny or stuffy nose, feeling of liquid running down the back of your throat, frequent throat clearing, and hoarseness. In this case, I would recommend trying the medication regimen suggested, which includes a tablet containing montelukast + levocetirizine, cough suppressant syrup before bed, and pantoprazole 40mg empty stomach in the morning. Additionally, you may want to try to reduce acid reflux by eating smaller, more frequent meals, waiting three to four hours after a meal before lying down, and elevating the head of the bed at night. If your symptoms do not improve, it's recommended to see a doctor and get a chest X-ray done to get a confirmed diagnosis.